# Imports

In [1]:
import folium
from folium.plugins import Draw, MarkerCluster

In [2]:
import sys
import os
from pathlib import Path
import pandas as pd
import networkx as nx
import json
# Opciones de visualización
import matplotlib.pyplot as plt


In [3]:
# Aseguramos que el directorio raíz del proyecto esté en el path
#project_root = Path().resolve().parent  # Ajusta si es necesario
#if str(project_root) not in sys.path:
#    sys.path.insert(0, str(project_root))

# Obtener ruta absoluta del directorio que contiene el notebook
notebook_dir = os.path.dirname(os.getcwd())  # sube un nivel desde /notebook
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)

# Selecionar Area

In [ ]:
lat, lon, zoom = 40.35, 0.40, 10
m = folium.Map(location=[lat, lon], zoom_start=zoom, tiles='OpenStreetMap')

folium.TileLayer(
    tiles='https://tiles.openseamap.org/seamark/{z}/{x}/{y}.png',
    attr='© OpenSeaMap contributors',
    name='Seamarks',
    overlay=True,
    control=True
).add_to(m)

folium.raster_layers.WmsTileLayer(
    url='https://ows.emodnet-bathymetry.eu/wms?',
    layers='emodnet:contours',
    fmt='image/png',
    transparent=True,
    version='1.3.0',
    attr='© EMODnet Bathymetry',
    name='Contornos (EMODnet)',
    overlay=True,
    control=True,
    opacity=1.0
).add_to(m)

draw = Draw(
    export=True,
    filename='selection.geojson',
    position='topright',
    draw_options={
        'polyline': False,
        'polygon': False,
        'circle': False,
        'marker': False,
        'circlemarker': False,
        'rectangle': True,
    },
    edit_options={'edit': False}
)
draw.add_to(m)

folium.LayerControl().add_to(m)
m


# Cargar zona de estudio

In [20]:
from generate_area.select_area import get_area_bounds

path_geojson = os.path.join("..", "..", "data", "raw", "selection_1.geojson")

In [21]:
# 2) Obtener límites
lat_min, lat_max, lon_min, lon_max = get_area_bounds(path_geojson)
print(f"lat_min = {lat_min:.6f}, lat_max = {lat_max:.6f}")
print(f"lon_min = {lon_min:.6f}, lon_max = {lon_max:.6f}")

lat_min = 37.865320, lat_max = 40.651460
lon_min = -0.098924, lon_max = 5.156147


# Generar malla

In [22]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from generate_area.generate_meh import generate_mesh_mercator

In [24]:
# 2.1 – Parámetros de la malla
paso_lon = 0.042   # espaciado en longitud (°)
N_lat    = 50      # número de intervalos en latitud

In [25]:
# 2.2 – Generar lista de puntos (lat, lon)
puntos_malla = generate_mesh_mercator(
    lat_min, lat_max,
    lon_min, lon_max,
    paso_lon,
    N_lat
)

In [27]:
# 2.3 – Crear GeoDataFrame de nodos
mesh_gdf = gpd.GeoDataFrame(
    {
        'latitude': [pt[0] for pt in puntos_malla],
        'longitude': [pt[1] for pt in puntos_malla]
    },
    geometry=[Point(lon, lat) for lat, lon in puntos_malla],
    crs='EPSG:4326'
)

print(f"Total nodos generados: {len(mesh_gdf)}")

Total nodos generados: 5355


## Visualizacion de malla

In [28]:
# 1) Centrar el mapa en el área de estudio
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2
m = folium.Map(location=[center_lat, center_lon], zoom_start=9)

# 2) Capa de símbolos náuticos (OpenSeaMap)
folium.TileLayer(
    tiles='https://tiles.openseamap.org/seamark/{z}/{x}/{y}.png',
    attr='© OpenSeaMap contributors',
    name='Seamarks',
    overlay=True,
    control=True
).add_to(m)

# 3) Capa de contornos batimétricos (EMODnet)
folium.raster_layers.WmsTileLayer(
    url='https://ows.emodnet-bathymetry.eu/wms?',
    layers='emodnet:contours',
    fmt='image/png',
    transparent=True,
    version='1.3.0',
    attr='© EMODnet Bathymetry',
    name='Contornos (EMODnet)',
    overlay=True,
    control=True,
    opacity=0.7
).add_to(m)

# 4) Dibujar cada nodo de la malla como un CircleMarker
for _, row in mesh_gdf.iterrows():
    folium.CircleMarker(
        location=(row.latitude, row.longitude),
        radius=2,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        weight=0
    ).add_to(m)

# 5) Control de capas
folium.LayerControl().add_to(m)

# 6) Mostrar el mapa (en Jupyter se renderiza automáticamente)
m

# Obtener profundidades

In [ ]:
import pandas as pd
from generate_area.depht_data_meh import batch_evaluate_navigability

In [30]:
# 3.1 – Parámetro de calado mínimo de la embarcación (en metros)
draft = 2.0

# 3.2 – Llamada al batch para obtener depth_avg y flag navigable
bathy_results = batch_evaluate_navigability(
    [(row.latitude, row.longitude) for _, row in mesh_gdf.iterrows()],
    draft
)

In [ ]:
# 3.3 – Convertir a DataFrame
df_bathy = pd.DataFrame(bathy_results)

# 3.4 – Unir con la malla original para conservar geometría
bathy_gdf = mesh_gdf.merge(df_bathy, on=['latitude', 'longitude'])

# 3.5 – Exportar resultados
df_bathy.to_csv('../../data/raw/malla/nodes_bathy.csv', index=False)

In [32]:
# (Opcional) Mostrar resumen
print(f"Nodos totales: {len(bathy_gdf)}")
print(f"Nodos navegables: {bathy_gdf['navigable'].sum()}")

Nodos totales: 5355
Nodos navegables: 3965


## Validación

In [26]:
import pandas as pd

# Cargar el CSV unificado
df = pd.read_csv('../../data/raw/malla/nodes_bathy.csv')

# 1) Resumen de valores nulos por columna
print("Valores nulos por columna:")
print(df.isna().sum())


Valores nulos por columna:
latitude     0
longitude    0
depth_avg    0
navigable    0
dtype: int64


# Obtener vientos

In [29]:
import pandas as pd
from generate_area.wind_data_meh import batch_evaluate_wind

In [30]:
# 4.1 – Cargar resultados de batimetría
df_bathy = pd.read_csv('../../data/raw/malla/nodes_bathy.csv')

In [31]:
# 4.2 – Definir parámetros de tu barco y periodo de interés
boat_data = {
    'max_wind_speed': 25.0,   # umbral en nudos
    # ...otros parámetros (e.g. polar, calado) si los requiere batch_evaluate_wind
}
start_date = '2025-06-01'
end_date   = '2025-06-01'

In [32]:
# 4.3 – Ejecutar batch para nodos navegables
wind_results = batch_evaluate_wind(
    df_bathy.to_dict(orient='records'),
    boat_data,
    start_date,
    end_date
)

In [36]:
# 4.4 – Crear DataFrame de resultados de viento
df_wind = pd.DataFrame(wind_results)

# 4.5 – Unir con batimetría para mantener todos los campos
df_bathy_wind = df_bathy.merge(
    df_wind[['latitude','longitude','time','wind_speed_10m','wind_direction_10m','navigable']],
    on=['latitude','longitude'],
    suffixes=('','_wind')
)

In [37]:
# 4.6 – Guardar salida
df_bathy_wind.to_csv('../../data/raw/malla/nodes_bathy_wind.csv', index=False)

## Validación

In [ ]:
import pandas as pd

# Cargar el CSV unificado
df = pd.read_csv('../../data/raw/malla/nodes_bathy_wind.csv')

# 1) Resumen de valores nulos por columna
print("Valores nulos por columna:")
print(df.isna().sum())


Valores nulos por columna:
latitude              0
longitude             0
depth_avg             0
navigable             0
time                  0
wind_speed_10m        0
wind_direction_10m    0
navigable_wind        0
dtype: int64


# Obtenemos los obstaculos

In [39]:
import pandas as pd
import json
from shapely.ops import unary_union
from shapely.geometry import mapping
from generate_area.nautical_constraints_meh import (
    load_forbidden_zones,
    mark_blocked_seamark,
    fetch_seamark_elements,
    fetch_coastline_elements,
    classify_seamarks,
    create_buoy_nodes,
    combine_mesh_and_buoys,
    compute_navigable_final
)

In [40]:
# 5.1 – Cargar resultados de batimetría + viento
df_bw = pd.read_csv('../../data/raw/malla/nodes_bathy_wind.csv')

In [41]:
# 5.2 – Cargar zonas prohibidas (seamarks + línea de costa)
tree = load_forbidden_zones(lat_min, lon_min, lat_max, lon_max, seguridad=0.01)
restricted_geoms = list(tree.geometries)

# 5.3 – Marcar los nodos bloqueados por seamarks/costa
df_bw['blocked_seamark'] = mark_blocked_seamark(df_bw, restricted_geoms, seguridad=0.01)

# 5.4 – Clasificar y crear nodos de boya
elements     = fetch_seamark_elements(lat_min, lon_min, lat_max, lon_max)
buoy_geoms   = classify_seamarks(elements)
buoy_df      = create_buoy_nodes(buoy_geoms)

ValueError: Tipo no soportado en buoy_geoms: <class 'shapely.geometry.linestring.LineString'>

In [ ]:
# 6 – Unificar todos los datos y computar navegabilidad final
nodes_df = combine_mesh_and_buoys(df_bw, buoy_df)
nodes_df['navigable_final'] = compute_navigable_final(nodes_df)
nodes_df.to_csv('../../data/raw/malla1/nodes_unified.csv', index=False)

c:\Users\aigut\OneDrive\Documentos\Sailing-route-generator-for-sailboats\tfm_deterministic_agent\generate_area\nautical_constraints_meh.py:183: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([mesh_df, buoy_df], ignore_index=True, sort=False)


In [ ]:
# 7 – Exportar zonas no navegables a GeoJSON
# Unir todos los polígonos de restricted_geoms en un único MultiPolygon
union_poly = unary_union(restricted_geoms)
# Crear FeatureCollection
features = [
    {"type":"Feature", "geometry": mapping(poly)}
    for poly in (union_poly.geoms if hasattr(union_poly, "geoms") else [union_poly])
]
with open('../../data/raw/malla/non_navigable_zones.json', 'w') as f:
    json.dump({"type":"FeatureCollection", "features": features}, f)

# Calculo de grafo ponderado

In [42]:
import pandas as pd
import json
import pickle
from shapely.geometry import shape, MultiPolygon
from generate_area.build_graph import build_weighted_graph

In [43]:
# 8.1 – Cargar ficheros de entrada
nodes_df   = pd.read_csv('../../data/raw/malla/nodes_unified.csv')
polar_df   = pd.read_csv('../../data/raw/malla/polar_diagram.csv')
with open('../../data/raw/malla/non_navigable_zones.json', 'r', encoding='utf-8') as f:
    zones_fc = json.load(f)

In [44]:
# 8.2 – Construir la geometría unida de zonas no navegables
geoms = [shape(feat['geometry']) for feat in zones_fc['features']]
if len(geoms) > 1:
    union_restr = MultiPolygon(geoms)
else:
    union_restr = geoms[0]

# 8.3 – Crear el grafo dirigido ponderado
G = build_weighted_graph(
    nodes_df,
    polar_df,
    union_restr,
    max_neighbors=32,
    neighbor_cells=3,
    alpha_time=1.0,
    beta_comfort=0.1
)

IndexError: single positional indexer is out-of-bounds

In [ ]:
# 8.4 – Guardar el grafo para uso posterior
with open('../../data/raw/malla/weighted_graph.pkl', 'wb') as f:
    pickle.dump(G, f)

print(f"Grafo creado: {G.number_of_nodes()} nodos, {G.number_of_edges()} aristas")

## Visualización de grafo